# Note to self
Next time start by:


Then:
- Check wind prof calcs




# Setup

In [1]:
import os
#Specify whether to use CHEESEHEAD or synthtic data 
os.environ['config'] = 'CHEESEHEAD'
# os.environ['config'] = 'synthetic'

from setup import * #Import setup module
from chad_funcs import *

# import pdb #Add to setup if I'm going to use this often

# #If using limited cases- full case list read in through setup module
# cases = pd.read_csv('../Inputs/cases_limited.csv', index_col = 'case') #dataframe of tower coordinates

In [2]:
# # # Reimport
# import importlib
# import chad_funcs
# importlib.reload(chad_funcs)
# from chad_funcs import*

# Functions

## Hrz Advection Wrapping Function

In [ ]:
#Hrz advection calcs
'''
Function that caculates horizontal advection of H and LE

Parameters:
- hrz_grad_tow: horizontal gradients for given tower
- WS: 
- step: step size for vertical profiles, defaule = 0.1 m

'''

def hrz_ad_calc(hrz_grad, WS, stab_meth, MO, dtindex, step = 0.1):
    
    H_adv = pd.DataFrame(index = dtindex, columns = towlist)
    LE_adv = pd.DataFrame(index = dtindex, columns = towlist)
    
    for tow in towlist:
        
        #Used to have in if statement but I think this should work for all towers, delete if statement later if not needed
        #if tc.loc[tow].z3 != 0: #All three-level towers 
        WS_prof = wind_prof(tow, stab_meth, WS[tow], MO[tow], dtindex, step, z_Umes = 'use_mes')        

        #H advection
        H_adv_step = prof_calcs(tow, hrz_grad['TA'], 'interp', dtindex, step).values*WS_prof.mul(-rho[tow]*cp*step, axis = 0) #Advection of sensible heat
        #Units W/m^2 = K/m                   [m/s]     [kg/m^3][J/kg*K][m]

        #LE advection
        LE_adv_step = prof_calcs(tow, hrz_grad['H2O'], 'interp', dtindex, step).values*WS_prof.mul(-step*L) #Advection of latent heat
        #Units: [W/m^2] =   [g/m^3/m]          [m/s]       [m]  [J/g]

        #Note: The /1000 in both advection equations is because gradients are in per km and need to be in per m

        # hrz_ad = (H_ad + LE_ad)
        #Bottom value on all profiles is set to 0 so some profiles with all missing data still have 1 real value so min_count = 2
        H_adv[tow] = H_adv_step.sum(axis = 1, min_count = 2)
        LE_adv[tow] = LE_adv_step.sum(axis = 1, min_count = 2)
                    
    
    return H_adv, LE_adv 
    
    #Deal with water vapor later #fixthis what is this??
    # md_w = e/(R*TA_K) #Molar density of water vapor [mol/m^3], used in SLE calc

## Vertical Advection Calc

In [4]:
#Vert H and LE advection calcs
def vert_ad_calc(tow, tracdat, w_tow, MO_tow, step = 0.1):

    towdat = tc.loc[tow]
    
    TA_mean = prof_calcs(tow, tracdat['TA'], 'mes', dtindex, step).mean(axis = 1)
    H2O_mean = prof_calcs(tow, tracdat['H2O'], 'mes', dtindex, step).mean(axis = 1)
    
    TA_diff = tracdat['TA'][towdat.top_lev, tow] - TA_mean
    H2O_diff = tracdat['H2O'][towdat.top_lev, tow] - H2O_mean
                   
    H_vert = TA_diff*w_tow*rho[tow]*cp
    LE_vert= H2O_diff*w_tow*L    
    
    return H_vert, LE_vert

## Profile Calcs

### Wind Profile

### Profiles of tracer (or tracer gradients)

# Setup this notebook

## Read in Data

In [5]:
#Full length
startdate = dt.datetime(2019, 6, 20, 0)
enddate = dt.datetime(2019, 10, 13, 23)
# startdate = dt.datetime(2019, 7, 1)
# enddate = dt.datetime(2019, 7, 2)

dtindex = readdata('dtindex', startdate, enddate)

#Read in data -potentially move into modules but that would make it challenging 
hrz_wind = readdata('hrz_wind', startdate, enddate) #Horizontal wind- includes WS_top, WD
MO = readdata('MO', startdate, enddate)

#Air density used in sensible head advection calc (hrz_ad_calc)
TA_K = readdata('TA', startdate, enddate)[30] + CtoK #Temperature in Kelvin
PA_top_Pa = readdata('PA_top', startdate, enddate)*1000 #kPa to Pa - probbably need to adjust for PFA
rho = mm*PA_top_Pa/TA_K/R #Dry air density at each tower [kg/m^3] **should probably use virtual temperature here** #fixthis

#Gap filling
for tow in towlist:
    rho[tow] = rho[tow].fillna(rho.mean(axis = 1)) 

# #Method dependent data (move into functions), could move into  #fixthis
w_son = readdata('w_son', startdate, enddate)
w_cont = readdata('w_cont', startdate, enddate)
w_cont_lai = readdata('w_cont_lai', startdate, enddate)

In [6]:
tracdat = {'TA': readdata('TA', startdate, enddate),
              'H2O': readdata('H2O', startdate, enddate)} 

# Run Calcs

## Horizontal Advection

In [7]:
H_hrz = make_multi_df([cases.index, towlist], ['case', 'tow'], dtindex)
LE_hrz = make_multi_df([cases.index, towlist], ['case', 'tow'], dtindex)

for case in cases.index:
    print(case)
    grad_method = cases['grad_method'].loc[case]
    interpdist = cases['interp_dist'].loc[case]
    interpdir = cases['interp_dir'].loc[case]
    stab_meth = cases['stability'].loc[case]
    kernel = cases['interp_kernel'].loc[case]

    if grad_method == 'gradavg':
        filepath = intermed_filepath + 'hrz_gradients/'+ grad_method + '.pickle'
        
    elif grad_method == 'interp':
        filepath = intermed_filepath + 'hrz_gradients/tracer' + grad_method + str(interpdist) + interpdir + kernel + '.pickle'
    
    # # Filepath for shortened time
    filepath = intermed_filepath + 'hrz_gradients/tracer' + grad_method + str(interpdist) + str(interpdir) \
     + str(kernel) +str(stab_meth) + startdate.strftime("%m%d")+ '-'+ enddate.strftime("%m%d")+'.pickle'
    
    with open(filepath, 'rb') as handle:
        hrz_grad = pickle.load(handle)
        
    H_hrz[case], LE_hrz[case] = hrz_ad_calc(hrz_grad, hrz_wind['WS_top'], stab_meth, MO, dtindex, step = 0.1)

A
B
C
D
E
F
G
H
I
J
K
L


### Export Output

In [8]:
filepath = output_filepath + 'adv_final/H_hrz_all_lai.pickle'

with open(filepath, 'wb') as handle:
    pickle.dump(H_hrz, handle)

    
filepath = output_filepath + 'adv_final/LE_hrz_all_lai.pickle'


with open(filepath, 'wb') as handle:
    pickle.dump(LE_hrz, handle)

## Vertical Advection

In [9]:
#Calc vert adv for all 30 m towers (fiture out shorter towers later)
vert_ad_all = make_multi_df([['H', 'LE'], towlist], ['var', 'tow'], dtindex)
step = 0.1
w_use = w_cont_lai 
stab_meth = None

for tow in tc.index:
    if tow == 'PFA' or tow == 'PFd' or tow == 'PFr':
        pass
    else:
        vert_ad_all['H', tow], vert_ad_all['LE', tow] = vert_ad_calc(tow, tracdat, w_use[tow], MO[tow], step)

### Export Output

In [10]:
# filepath = output_filepath + 'adv_final/vert_ad_cont_lai.pickle'


# with open(filepath, 'wb') as handle:
#     pickle.dump(vert_ad_all, handle)

# Testing/plotting

# Archive/need to figure out what this stuff does

In [11]:
# #color list
# colorlist = ["xkcd:ocean blue", "xkcd:terracotta", "xkcd:blue green","xkcd:grape",  "xkcd:grey",  
#              "xkcd:forrest green","xkcd:magenta", "xkcd:grey blue", "xkcd:dark yellow","xkcd:coral",
#               "xkcd:sky blue", "xkcd:light orange","xkcd:reddish brown", "xkcd:dark lavender",  "xkcd:light olive",
#              "xkcd:puke yellow", "xkcd:light cyan", "xkcd:orchid",  "xkcd:periwinkle blue"]
# mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colorlist) 

In [12]:
# tow = 'PFm'
# dir_range = np.arange(0, 370, 10)
# w_grouped1 = w_cont[tow].groupby(pd.cut(hrz_wind['WD'][tow], dir_range)).mean()
# w_grouped2= w_son[tow].groupby(pd.cut(hrz_wind['WD'][tow], dir_range)).mean()
# plt.plot(dir_range[0:-1], w_grouped1)
# plt.plot(dir_range[0:-1], w_grouped2)

In [13]:
# tow = 'PFp'
# dir_range = np.arange(0, 370, 10)
# flux_grouped1 = H_hrz[2][tow].groupby(pd.cut(hrz_wind['WD'][tow], dir_range)).mean()
# flux_grouped2= LE_hrz[2][tow].groupby(pd.cut(hrz_wind['WD'][tow], dir_range)).mean()
# plt.plot(dir_range[0:-1], flux_grouped1)
# plt.plot(dir_range[0:-1], flux_grouped2)

In [14]:
# w_son[tow].groupby(hrz_wind['WD'][tow])

In [15]:
# # dat_use = w_cont
# dat_use = H_hrz[2]
# for tow in towlist:
#     plt.plot(dat_use[tow].groupby(dat_use.index.hour).mean(), label = tow);
# plt.legend()

In [16]:
# plt.plot(vert_ad_all_cont['H', 'PFg'].groupby(vert_ad_all_son.index.hour).mean())

In [17]:
# tow = 'PFg'
# plt.hist(vert_ad_all_son['H', tow], bins = 50);

## Labeled code chunks

### Old wind profile method

In [18]:
''' Can probably get rid of this later, maybe save and compare wind profiles with 
stability correction to wind profiles from Ustar without using ratio to correct'''

# #Formetly in Wind profile calc function, when using Ustar from data, now using ration approach which should give same result
# WS_calc = WS_top_calc(tow)
# WS_calc_rat = hrz_wind['WS_top'][tow]/WS_calc
# u_of_h = hrz_wind['Ustar'][tow]/k*np.log((h_veg-d)/z_m)*WS_calc_rat #wind speed at top of canopy


# #Function to calc wind speed at top of tower using ustar
# def WS_top_calc(tow):
#     k = 0.4 #von Karman constant
    

#     h_tow = tc.z1.loc[tow] #Tower height
#     h_veg = tc.veg_h.loc[tow] #Vegetation height
#     z0 = 0.1*h_veg #roughness length,might be good to look at plant data to better estimate
#     d = tc.d.loc[tow] #Displacement height
    
#     WS_calc = (hrz_wind['Ustar'][tow]/k)*np.log((h_tow - d)/(z0))
#     return WS_calc


' Can probably get rid of this later, maybe save and compare wind profiles with \nstability correction to wind profiles from Ustar without using ratio to correct'

### Wind profile testing

In [19]:
#Assessing wind profile calculations
# WS_calc = pd.DataFrame(index = dtindex)
# for tow in towlist:
#     WS_calc[tow]= WS_top_calc('PFb', d_rat = 0.65)
# WS_real = hrz_wind.loc['WS_top']
# RMSE = (((WS_calc - WS_real)**2).mean()**(1/2))#.mean()

# wind_rat_all = WS_calc/WS_real
# wind_rat_di = wind_rat_all.groupby([wind_rat_all.index.hour, wind_rat_all.index.minute]).mean()
# xvals = wind_rat_di.index.get_level_values(0).values + wind_rat_di.index.get_level_values(1).values/60
# wind_rat_di.set_index(xvals, inplace = True)
# wind_rat_di.plot(figsize = (15, 5))
# plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
# plt.xlabel('hour')
# plt.ylabel('WS_calc/WS_mes')

# #Attempt to calculate d from WS and Ustar - didn't work well, got huge numbers
# d_calc = pd.DataFrame(index = dtindex)

# for tow in towlist:
#     z = tc.z1.loc[tow]
#     z0 = tc.veg_h.loc[tow]*0.1
#     Ustar_pos = hrz_wind.loc['Ustar'][tow].where(hrz_wind.loc['Ustar'][tow]>0)
#     d_calc[tow] = z - z0*np.exp(hrz_wind.loc['WS_top'][tow]*k/Ustar_pos)
#     # d_calc[tow] = np.exp(hrz_wind.loc['WS_top'][tow]*k/Ustar_pos)

### Save output data

In [20]:
#Saving output data
# filepath = '../Outputs/Hrz_adv/vardirtest_LE_adv'

# with open(filepath, 'wb') as handle:
# pickle.dump(LE_ad, handle)

### Seasonal transition dates

In [21]:
# #Approximate dates of transition between dry, wet and windy seasons

# date1 = dt.datetime(2019, 7, 1)
# date2 = dt.datetime(2019, 7, 12)
# date3 = dt.datetime(2019, 8, 20)
# date4 = dt.datetime(2019, 7, 10)

## Unblabled/unorganized code chunks

In [22]:
# # Code to rename vert_wind columns - from var names to tow names
# # Don't need because I resaved the pickel with new column names
# tow_names =  dict(zip(tc.wind_var[1:], tc.index[1:])) #Dictionary to relate 
# vert_wind = vert_wind.rename(columns = tow_names)
# vert_wind['PFA'] = np.nan #Add column for PFA since I don't have sonic data for it - maybe see if I can get that
# vert_wind = vert_wind[towlist]
# vert_wind.to_pickle('../Inputs/w_son.pickle') #<-here's how I renamed it

In [23]:
#Filepath for shortened time saved data
# filepath = '../intermed_data/hrz_gradients/vardirtest_'+ str(interpdist) + interpdir \
# + startdate.strftime("%m%d")+ '-'+ enddate.strftime("%m%d")+'.pickle'

In [24]:
# #Function to read in universal data
# #Reads in all data unless start and end date are specified
# def read_unidata(startdate = dt.datetime(2019, 6, 1), enddate = dt.datetime(2019, 11, 1)):
#     unidata_paths = {
#         'TA':'../Inputs/TA_orig_all.pickle',
#         'H2O':'../Inputs/H2O_orig_all.pickle',
#         'hrz_wind': '../Inputs/wind_dat.pickle',
#         'PA_top': '../Inputs/PA_top.pickle'}
#     unidata = {} #Save all data in unidata in unidata 
#     for var in unidata_paths.keys():
#         if var == 'PA_top':
#             #No levels for PA, select start to end date
#             unidata[var] = pd.read_pickle(unidata_paths[var]).loc[startdate:enddate] 
#         else:
#             #Select all levels, start to end date, all tows
#             unidata[var] = pd.read_pickle(unidata_paths[var]).loc[idx[:, startdate:enddate], :] 
            
#     unidata['dtindex'] = unidata['TA'].loc[2].index #Datetime index for all datasets
#     return unidata
        
#Function for creating multiindexed dataframes

In [25]:
# #Calc hrz adv for all 30 m tows (fiture out shorter towers later)
# H_vert_d = pd.DataFrame()
# LE_vert_d = pd.DataFrame()

# for tow in tc.index:
#     if tc.loc[tow].z1 == 30:
#         if tow == 'PFA':
#             pass
#         else:
#             H_vert, LE_vert = vert_ad_d_calc(tow)
#             H_vert_d[tow] = H_vert
#             LE_vert_d[tow] = LE_vert
        
# vert_ad_d = pd.concat([H_vert_d, LE_vert_d], keys = ['H', 'LE'])   

In [26]:
# for tow in tc.index:
#     if tow in ['PFA', 'PFc', 'PFd', 'PFl', 'PFr', 'PFs']:
#         pass
#     else:
#         for var in ['LE', 'H']:
#             if var == 'LE':
#                 max_bin = 2000
#             else:
#                 max_bin = 500
#             # f1 = plt.figure()
#             min_bin = -max_bin

#             binwidth = max_bin//25

#             plt.hist(vert_ad[tow].loc[var], bins=range(min_bin, max_bin + binwidth, binwidth), density = True);
#             plt.hist(vert_ad_d[tow].loc[var], bins = range(min_bin, max_bin + binwidth, binwidth), alpha = 0.7, density = True);

#             plt.xlabel(var + ' [$W/m^2$]')
#             plt.ylabel('Frequency')

#             plt.savefig('./Outputs/d_v_0_vert_ad/' + tow + '_' + var)
#             plt.cla()

In [27]:
# tower = 'PFb'

# #Calc vert T diff (only works for 30 m towers(only works for 30 m towers
# TA_30 = TA.loc[30][tower]
# TA_10 = TA.loc[10][tower]
# TA_2 = TA.loc[2][tower]
# TA_0 = TA_2 - (TA_10 - TA_2)*(2/8) #Surface temp
# TA_diff = TA_0 - TA_30 

# #Cut out upper and lower 10% of TA_diff
# l_cutoff = np.nanpercentile(TA_diff, 10)
# u_cutoff = np.nanpercentile(TA_diff, 90)

# #TA_diff = TA_diff.where(TA_diff>l_cutoff)#.where(TA_diff<u_cutoff)
# plt.hist(TA_diff, bins = 50)
# plt.axvline(x = l_cutoff, color = 'r')
# plt.axvline(x = u_cutoff, color = 'r')
# plt.savefig('../Outputs/Poster_plots/PFh_TA_diff_hist.png')

In [28]:
# plt.plot(TA_diff.groupby(TA_diff.index.hour).mean())

In [29]:
# tower = 'PFb'
# H_ad_use = vert_ad.loc['H'][tower]

# vert_H_l_cut = np.nanpercentile(H_ad_use, 5)
# vert_H_u_cut = np.nanpercentile(H_ad_use, 95)

# # vert_H_cut = H_ad_use.where(H_ad_use >vert_H_l_cut).where(H_ad_use <vert_H_u_cut)

# print(vert_H_l_cut, vert_H_u_cut)

# plt.hist(H_ad_use, bins = 50)
# plt.axvline(x = vert_H_l_cut, color = 'r')
# plt.axvline(x = vert_H_u_cut, color = 'r')


In [30]:

# plt.hist(vert_ad.loc['H']['PFb'], bins = 100, density = True);
# plt.hist(vert_ad.loc['LE']['PFb'], bins = 100, density = True, alpha = 0.7);

# print(vert_ad.loc['H']['PFg'].mean(), vert_ad.loc['LE']['PFg'].mean() )

In [31]:
# time_index = np.arange(0, 24, 0.5)

In [32]:
# plt.hist(wind_tt.values.flatten(), bins = 50);

In [33]:
# vert_H = vert_ad.loc['H'][startdate:enddate]
# wind_tt = vert_wind[vert_H.columns]#vert wind for tall towers

# plt.scatter(wind_tt, vert_H/wind_tt);

In [34]:
# tower = 'PFm'
# TA_2 = TA.loc[2][startdate:enddate][vert_H.columns]
# TA_30 = TA.loc[30][startdate:enddate][vert_H.columns]
# TA_0 = TA_2 - (TA_10 - TA_2)*(2/8)
# TA_diff = TA_30-TA_0

# TA_use = TA_diff[tower]

# vert_LE = vert_ad.loc['LE'][startdate:enddate]
# wind_tt = vert_wind[vert_H.columns][tower]#vert wind for tall towers

# bad_indices = np.isnan(wind_tt) | np.isnan(TA_use)
# good_indices = ~bad_indices
# good_wind = wind_tt[good_indices]
# good_TA = TA_use[good_indices]

# plt.hist2d(good_wind, good_TA, bins = 50);


In [35]:
# plt.scatter(hrz_ad.loc['H'][vert_H.columns], vert_ad.loc['H'])

In [36]:
#Note: removed changing line color by date - no decernable seasonal pattern

# cmap = plt.get_cmap('viridis').reversed()
# norm = plt.Normalize(0, len(vert_ad.loc['H'].index.date))

In [37]:
# cutoff = 50
# df_precut = vert_ad.loc['H']['PFe']

# df = df_precut.where(df_precut<cutoff).where(df_precut>-cutoff)
# for day in vert_ad.loc['H'].index.date:

#     plt.plot(time_index, df[np.in1d(df.index.date, [day])], linewidth = 0.1, color = 'lightslategray')
#     plt.xlabel('Hour')
#     plt.ylabel('Vertical Sensible Heat Advective Flux $W/m^2$')
    
# plt.ylim(-cutoff, cutoff)

# # plt.ylim(-250, 250)
# plt.plot(time_index, df.groupby(df.index.time).mean(), 'k', linewidth = 3)

# plt.savefig('../Outputs/Poster_plots/H_vert_PFe_daily_cut.png')

In [38]:
# cutoff = 50
# df_precut = vert_ad.loc['LE']['PFe']


# df = df_precut.where(df_precut<cutoff).where(df_precut>-cutoff)
# for day in vert_ad.loc['LE'].index.date:
    
#     plt.plot(time_index, df[np.in1d(df.index.date, [day])], linewidth = 0.2, color = 'lightslategray')
#     plt.xlabel('Hour')
#     plt.ylabel('Vertical Latent Heat Advective Flux $W/m^2$')
    
# # plt.ylim(-500, 500)
# plt.ylim(-cutoff, cutoff)
# plt.plot(time_index, df.groupby(df.index.time).mean(), 'k', linewidth = 3)

# plt.savefig('../Outputs/Poster_plots/LE_vert_PFe_daily_cut.png')

In [39]:
# df = hrz_ad.loc['H']['PFe']

# for day in vert_ad.loc['H'].index.date:
    
#     linecolor = cmap(norm(i))
#     plt.plot(time_index, df[np.in1d(df.index.date, [day])], linewidth = 0.1, color = 'lightslategray')
#     plt.xlabel('Hour')
#     plt.ylabel('Horizontal Sensible Heat Advective Flux $W/m^2$')
    
    
    
    
# plt.ylim(-8, 8)
# plt.plot(time_index, df.groupby(df.index.time).mean(), 'k', linewidth = 3)

# plt.savefig('../Outputs/Poster_plots/H_hrz_PFe_daily_all.png')

In [40]:
# df = hrz_ad.loc['LE']['PFe']

# for day in vert_ad.loc['LE'].index.date:
    
#     linecolor = cmap(norm(i))
#     plt.plot(time_index, df[np.in1d(df.index.date, [day])], linewidth = 0.1, color = 'lightslategray')
#     plt.xlabel('Hour')
#     plt.ylabel('Horizontal Latent Heat Advective Flux $W/m^2$')
    
    
    
# plt.ylim(-15, 15)
# plt.plot(time_index, df.groupby(df.index.time).mean(), 'k', linewidth = 3)

# plt.savefig('../Outputs/Poster_plots/LE_hrz_PFe_daily_all.png')

In [41]:
# LE_vert_day = vert_ad.loc['LE'].groupby(vert_ad.loc['LE'].index.hour).mean()
# plt.plot(LE_vert_day);
# plt.ylabel('Calculated Vertical Advection of Latent Heat [$W/m^2$]')
# plt.xlabel('Hour')
# plt.savefig('../Outputs/Poster_plots/LE_vert_ad_all.png')

In [42]:
# LE_vert_day = vert_ad.loc['H'].groupby(vert_ad.loc['H'].index.hour).mean()
# plt.plot(LE_vert_day);
# plt.ylabel('Calculated Vertical Advection of Sensible Heat [$W/m^2$]')
# plt.xlabel('Hour')
# plt.savefig('../Outputs/Poster_plots/H_vert_ad_all.png')

In [43]:
# LE_vert_day = hrz_ad.loc['H'].groupby(hrz_ad.loc['H'].index.hour).mean()
# plt.plot(LE_vert_day);
# plt.ylabel('Calculated Horizontal Advection of Sensible Heat [$W/m^2$]')
# plt.xlabel('Hour')
# plt.savefig('../Outputs/Poster_plots/H_hrz_ad_all.png')

In [44]:
# LE_vert_day = hrz_ad.loc['LE'].groupby(hrz_ad.loc['LE'].index.hour).mean()
# plt.plot(LE_vert_day);
# plt.ylabel('Calculated Horizontal Advection of Latent Heat [$W/m^2$]')
# plt.xlabel('Hour')
# plt.savefig('../Outputs/Poster_plots/LE_hrz_ad_all.png')

In [45]:
# def day_cyc(dat):
#     dat_day = dat.groupby(dat.index.time).mean()
#     return dat_day

In [46]:
# tower = 'PFi'

# filepath = tc.org_path.loc[tower]
# df = pd.read_csv(filepath)

# df = df.applymap(lambda x: np.nan if x == -9999 else x)

# dt_ts = pd.to_datetime(df['TIMESTAMP_START'], format = '%Y%m%d%H%M') 
# df.set_index(dt_ts, inplace = True)


# startdate = dt.datetime(2019, 6, 20, 1)
# enddate = dt.datetime(2019, 10, 14, 23)
# df = df[startdate:enddate]

# NETRAD = df.NETRAD_1_1_1
# LE_turb = df.LE_1_1_1
# H_turb = df.H_1_1_1
# G = df. G_1_1_1

# LE_ad_hrz = hrz_ad.loc['LE'][tower]
# H_ad_hrz = hrz_ad.loc['H'][tower]

# LE_ad_vert = vert_ad.loc['LE'][tower]
# H_ad_vert = vert_ad.loc['H'][tower]

# E_bal = NETRAD - LE_turb - H_turb - G - LE_ad_hrz - H_ad_hrz - LE_ad_vert - H_ad_vert

# E_out = LE_turb + H_turb + G + LE_ad_hrz + H_ad_hrz + LE_ad_vert + H_ad_vert

# E_out_no_ad = LE_turb + H_turb + G

In [47]:
# plt.plot(np.cumsum(day_cyc(NETRAD)))
# # plt.plot(day_cyc(LE_turb))
# # plt.plot(day_cyc(H_turb))
# # plt.plot(day_cyc(G))
# # plt.plot(day_cyc(LE_ad_hrz))
# # plt.plot(day_cyc(H_ad_hrz))
# # plt.plot(day_cyc(LE_ad_vert))
# # plt.plot(day_cyc(H_ad_vert))

# plt.plot(np.cumsum(day_cyc(E_out_no_ad)))
# plt.plot(np.cumsum(day_cyc(E_out)))

# # plt.legend()


In [48]:
# # time_ind = hrz_ad.loc['H'].groupby(hrz_ad.loc['H'].index.time).mean().index
# time_ind = np.arange(0, 24, 0.5)

# fig, axs = plt.subplots(3,4, figsize = [13, 8], sharex=True, sharey=True)
# for i in range(0,3):
#     for j in range(0,4):
#         tower = hrz_ad.columns[i*4+j +1]
        
        
#         filepath = tc.org_path.loc[tower]
#         df = pd.read_csv(filepath)

#         df = df.applymap(lambda x: np.nan if x == -9999 else x)

#         dt_ts = pd.to_datetime(df['TIMESTAMP_START'], format = '%Y%m%d%H%M') 
#         df.set_index(dt_ts, inplace = True)


#         startdate = dt.datetime(2019, 6, 20, 1)
#         enddate = dt.datetime(2019, 10, 14, 23)
#         df = df[startdate:enddate]

#         NETRAD = df.NETRAD_1_1_1
#         LE_turb = df.LE_1_1_1
#         H_turb = df.H_1_1_1
#         G = df. G_1_1_1

#         LE_ad_hrz = hrz_ad.loc['LE'][tower]
#         H_ad_hrz = hrz_ad.loc['H'][tower]

#         LE_ad_vert = vert_ad.loc['LE'][tower]
#         H_ad_vert = vert_ad.loc['H'][tower]

#         E_bal = NETRAD - LE_turb - H_turb - G - LE_ad_hrz - H_ad_hrz - LE_ad_vert - H_ad_vert

#         E_out = LE_turb + H_turb + G + LE_ad_hrz + H_ad_hrz + LE_ad_vert + H_ad_vert

#         E_out_no_ad = LE_turb + H_turb + G
        
#         E_out_hrz_only = LE_turb + H_turb + G + LE_ad_hrz + H_ad_hrz
        
        
        
#         # axs[i, j].plot((np.cumsum(day_cyc(NETRAD).values)))
#         # axs[i, j].plot((np.cumsum(day_cyc(E_out).values)))
#         # axs[i, j].plot((np.cumsum(day_cyc(E_out_no_ad).values)))
#         # axs[i, j].plot((np.cumsum(day_cyc(E_out_hrz_only).values)))
        
#         a = axs[i, j].plot(time_ind, day_cyc(NETRAD).values, color = 'coral')
#         # b = axs[i, j].plot(time_ind, day_cyc(E_out).values, color = 'olivedrab')
#         c = axs[i, j].plot(time_ind, day_cyc(E_out_no_ad).values, color = 'cornflowerblue')
        
        
#         # axs[i, j].plot(day_cyc(E_out_hrz_only).values)
        
# fig.supxlabel('Hour')
# fig.supylabel('Energy Flux [$W/m^2$]')
# fig.legend([a, b, c], labels = ['Net radiation', 'Turbulent, ground and advective heat fluxes', 'Turbulent and ground heat flux only'], loc="upper right")

# fig.savefig('../Outputs/Poster_plots/diurnal_ebal_alltow.png')

In [49]:
# err_df = pd.DataFrame(columns = ['gap_change_med', 'gap_change_mean', 'gap_change_med_cut', 'gap_change_mean_cut'])#,
#                                 # 'gap_change_hrz_ad_only_med', 'gap_change_hrz_ad_only_med'])
# for tower in vert_ad.columns:
#     filepath = tc.org_path.loc[tower]
#     df = pd.read_csv(filepath)

#     df = df.applymap(lambda x: np.nan if x == -9999 else x)

#     dt_ts = pd.to_datetime(df['TIMESTAMP_START'], format = '%Y%m%d%H%M') 
#     df.set_index(dt_ts, inplace = True)


#     startdate = dt.datetime(2019, 6, 20, 1)
#     enddate = dt.datetime(2019, 10, 14, 23)
#     df = df[startdate:enddate]

#     NETRAD = df.NETRAD_1_1_1
#     LE_turb = df.LE_1_1_1
#     H_turb = df.H_1_1_1
#     G = df. G_1_1_1

#     LE_ad_hrz = hrz_ad.loc['LE'][tower]
#     H_ad_hrz = hrz_ad.loc['H'][tower]

#     LE_ad_vert = vert_ad.loc['LE'][tower]
#     H_ad_vert = vert_ad.loc['H'][tower]

#     LE_ad_vert_cut = LE_ad_vert.where(LE_ad_vert>np.nanpercentile(LE_ad_vert, 10)).where(LE_ad_vert<np.nanpercentile(LE_ad_vert, 90))
#     H_ad_vert_cut = H_ad_vert.where(H_ad_vert>np.nanpercentile(H_ad_vert, 10)).where(H_ad_vert<np.nanpercentile(H_ad_vert, 90))

#     E_bal_org = NETRAD - LE_turb - H_turb - G

#     E_bal_new = NETRAD - LE_turb - H_turb - G - LE_ad_hrz - H_ad_hrz - LE_ad_vert - H_ad_vert

#     E_bal_new_cut = NETRAD - LE_turb - H_turb - G - LE_ad_hrz - H_ad_hrz - LE_ad_vert_cut - H_ad_vert_cut
    
#     E_bal_hrz = NETRAD - LE_turb - H_turb - G - LE_ad_hrz - H_ad_hrz

#     # E_out = LE_turb + H_turb + G + LE_ad_hrz + H_ad_hrz + LE_ad_vert + H_ad_vert

#     # E_out_no_ad = LE_turb + H_turb + G


#     err_org = E_bal_org/NETRAD
#     err_new = E_bal_new/NETRAD
#     err_new_cut = E_bal_new_cut/NETRAD
#     err_hrz_ad_only = E_bal_hrz/NETRAD

#     # plt.hist(err_new, bins = 50, density = True);
#     # plt.hist(err_org, bins = 50, alpha = .5, density = True);

#     err_change = np.abs(err_org) - np.abs(err_new)
#     err_change_cut = np.abs(err_org) - np.abs(err_new_cut)
#     err_change_hrz_only = np.abs(err_org) - np.abs(err_hrz_ad_only)
#     # plt.hist(err_change, bins = 100)
#     err_df.loc[tower] = [err_change.median(), err_change.mean(), err_change_cut.median(), err_change_cut.mean()]#,
#                         # err_change_hrz_only.median(), err_change_hrz_only.mean()]



In [50]:
#Example T profile plot

# tower = 'PFn'
# TA_2 = TA.loc[2][tower].mean()
# TA_10 = TA.loc[10][tower].mean()
# TA_30 = TA.loc[30][tower].mean()
# TA_0 = TA_2 - (TA_10 - TA_2)*(2/8)

# plt.figure(figsize = (8,12))

# plt.plot(TA_2, 2, marker='o', color = 'k')
# plt.plot(TA_10, 10, marker='o',color = 'k')
# plt.plot(TA_30, 30, marker='o', color = 'k')

# plt.plot([TA_0, TA_10], [0, 10], linestyle = '--', color = 'k')
# plt.plot([TA_10, TA_30], [10, 30], linestyle = '--', color = 'k')

# plt.xlabel('Air temperature (degree C)')
# plt.ylabel('Hight (m)')

# plt.savefig('./Outputs/Poster_plots/PFn_TA_prof.png')

In [51]:
#Advection time series

# tower = 'PFb'
# # plt.plot(vert_ad.loc['H'][tower])
# fig, axs = plt.subplots(4)
# axs[0].plot(hrz_ad.loc['H'][tower])
# axs[1].plot(hrz_ad.loc['LE'][tower])
# axs[2].plot(vert_ad.loc['H'][tower])
# axs[3].plot(vert_ad.loc['LE'][tower])
# fig.tight_layout()
# plt.savefig('./Outputs/Poster_plots/PFb_ad_all.png')

In [52]:
# H_day = hrz_ad.loc['H'].groupby(hrz_ad.loc['H'].index.hour).mean()
# plt.plot(H_day)

In [53]:
#Old version - for NSEW gradients

# #Hrz advection calcs
# #Function that caculates horizontal advection of H and LE
# #Parameters:
# #list parameters here

# def hrz_ad_calc(tower, step = 0.1):

#     WS_prof, T_grad_NS, T_grad_EW, H2O_grad_NS, H2O_grad_EW = prof_calcs(tower, step)
    
#     TA_K = TA.loc[30][tower] + CtoK
    
#     PA_Pa = PA_top[tower]*1000 #kPa to Pa **this is at top of tower(~400m), need PA val for 30m and below**
#     rho = mm*PA_Pa/TA_K/R #Dry air density at each tower [kg/m^3] **should probably use virtual temperature here** 
#     z = step #np.asarray(tc.loc[tower].z1) #Height of each tower

#     SN_wind = WS_prof.mul(np.sin(hrz_wind.loc['WD'][tower]*np.pi/180), axis = 0)
#     WE_wind = WS_prof.mul(np.cos(hrz_wind.loc['WD'][tower]*np.pi/180), axis = 0)
    
#     #H advection
#     H_SN = T_grad_NS*SN_wind.mul(-rho*z*cp/2000, axis = 0) #Advection of sensible heat by southerly wind [W/m^2]
#     H_WE = T_grad_EW*WE_wind.mul(-rho*z*cp/2000, axis = 0) #Advection of sensible heat by westerly wind [W/m^2]
#     H_ad = H_SN + H_WE #Total advection

#     #LE advection
#     LE_SN = H2O_grad_NS*SN_wind.mul(-z*L/2000) #Advection of latend heat by southerly wind [W/m^2]
#     LE_WE = H2O_grad_EW*WE_wind.mul(-z*L/2000 )#Advection of latent hear by westerly wind [W/m^2]
#     LE_ad = (LE_SN + LE_WE)#*rho/rho #LE advection (the *rho/rho just makes it a df with the same tower and timestamp as indices)


#     # hrz_ad = (H_ad + LE_ad)
#     H_ad_tot = H_ad.sum(axis = 1)
#     LE_ad_tot = LE_ad.sum(axis = 1)
    
#     return H_ad_tot, LE_ad_tot
#     #Deal with water vapor later
#     # md_w = e/(R*TA_K) #Molar density of water vapor [mol/m^3], used in SLE calc